Question 1: (Home automation simulation)

In [ ]:
import time
import sqlite3
import paho.mqtt.client as mqtt
import json
from datetime import datetime

TEMPERATURE_THRESHOLD = 30.0  
HUMIDITY_THRESHOLD = 80.0     
DB_NAME = 'sensor_data.db'
MQTT_BROKER = 'mqtt.eclipseprojects.io'
MQTT_TOPIC_PUBLISH = 'home/sensor/temperature'
MQTT_TOPIC_SUBSCRIBE = 'home/alert/notification'
SENSOR_INTERVAL = 60  

def read_sensor():
    return {
        'temperature': 25.0 + (5 * (time.time() % 10) / 10),  
        'humidity': 45.0 + (40 * (time.time() % 10) / 10),  
        'timestamp': datetime.now().isoformat()
    }

def db():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS sensor_readings (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            temperature REAL NOT NULL,
            humidity REAL NOT NULL,
            timestamp TEXT NOT NULL,
            alert_sent INTEGER DEFAULT 0
        )
    ''')
    conn.commit()
    conn.close()

def store_reading(temperature, humidity, timestamp):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute('''
        INSERT INTO sensor_readings (temperature, humidity, timestamp)
        VALUES (?, ?, ?)
    ''', (temperature, humidity, timestamp))
    conn.commit()
    conn.close()

def on_connect(client, userdata, flags, rc):
    print(f"Connected with result code {rc}")
    client.subscribe(MQTT_TOPIC_SUBSCRIBE)

def on_message(client, userdata, msg):
    print(f"Received message on {msg.topic}: {msg.payload.decode()}")

mqtt_client = mqtt.Client()
mqtt_client.on_connect = on_connect
mqtt_client.on_message = on_message
mqtt_client.connect(MQTT_BROKER, 1883, 60)
mqtt_client.loop_start()

def send_alert(temperature, humidity):
    alert_message = {
        "alert": "Temperature/Humidity Alert",
        "message": f"Temperature {temperature}°C exceeds threshold {TEMPERATURE_THRESHOLD}°C" if temperature > TEMPERATURE_THRESHOLD else f"Humidity {humidity}% exceeds threshold {HUMIDITY_THRESHOLD}%",
        "timestamp": datetime.now().isoformat(),
        "values": {
            "temperature": temperature,
            "humidity": humidity
        }
    }
    
    mqtt_client.publish(MQTT_TOPIC_PUBLISH, json.dumps(alert_message))
    print(f"Alert sent: {alert_message}")

def monitor_sensors():
    db()
    
    while True:
        try:
            data = read_sensor()
            temp = data['temperature']
            hum = data['humidity']
            ts = data['timestamp']
            
            store_reading(temp, hum, ts)
            print(f"Stored reading: Temp={temp}°C, Hum={hum}%")
            
            mqtt_client.publish(MQTT_TOPIC_PUBLISH, json.dumps(data))
            
            if temp > TEMPERATURE_THRESHOLD or hum > HUMIDITY_THRESHOLD:
                send_alert(temp, hum)
            
            time.sleep(SENSOR_INTERVAL)
            
        except Exception as e:
            print(f"Error in monitoring: {e}")
            time.sleep(10)  
            
if _name_ == "_main_":
    monitor_sensors()

Question 2: (Monitoring Industrial performance)

In [ ]:
#include <iostream>
#include <vector>
#include <numeric>
#include <curl/curl.h>

double read_sensor() {
    return 25.0 + (rand() % 1000) / 100.0; // Simulated temp between 25 and 35°C
}

double calculate_average(const std::vector<double>& data) {
    return std::accumulate(data.begin(), data.end(), 0.0) / data.size();
}

bool send_data_secure(double value) {
    CURL* curl = curl_easy_init();
    if (!curl) return false;

    std::string postFields = "sensor=temperature&value=" + std::to_string(value);

    curl_easy_setopt(curl, CURLOPT_URL, "https://your-secure-server.com/api/data");
    curl_easy_setopt(curl, CURLOPT_POSTFIELDS, postFields.c_str());
    curl_easy_setopt(curl, CURLOPT_SSL_VERIFYPEER, 1L); // Validate SSL cert
    curl_easy_setopt(curl, CURLOPT_CAINFO, "/path/to/ca-bundle.crt"); // Your CA cert file

    CURLcode res = curl_easy_perform(curl);
    curl_easy_cleanup(curl);

    return res == CURLE_OK;
}

int main() {
    std::vector<double> buffer;
    const int sampleSize = 10;

    while (true) {
        double value = read_sensor();
        buffer.push_back(value);

        if (buffer.size() >= sampleSize) {
            double avg = calculate_average(buffer);
            std::cout << "Sending avg temperature: " << avg << "°C" << std::endl;

            if (!send_data_secure(avg)) {
                std::cerr << "Failed to send data!" << std::endl;
            }

            buffer.clear(); // reset for next window
        }

        std::this_thread::sleep_for(std::chrono::seconds(1));
    }

    return 0;
}

Question 3:

In [ ]:
import time
import random
import json
from datetime import datetime
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib
import requests


class SoilMoistureSensor:
    def read_moisture(self):
        return random.uniform(20, 80)

    def read_temperature(self):
        return random.uniform(10, 35)


class IrrigationPredictor:
    def _init_(self):
        self.model = None
        self.load_or_train_model()

    def load_or_train_model(self):
        try:
            self.model = joblib.load('irrigation_model.pkl')
            print("Loaded pre-trained model")
        except FileNotFoundError:
            self.train_model()

    def train_model(self):
        data = {
            'moisture': [random.uniform(20, 80) for _ in range(1000)],
            'temperature': [random.uniform(10, 35) for _ in range(1000)],
            'hour_of_day': [random.randint(0, 23) for _ in range(1000)],
            'needs_irrigation': [random.randint(0, 1) for _ in range(1000)]
        }

        df = pd.DataFrame(data)

        X = df[['moisture', 'temperature', 'hour_of_day']]
        y = df['needs_irrigation']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        self.model = RandomForestClassifier(n_estimators=100)
        self.model.fit(X_train, y_train)

        y_pred = self.model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Model trained with accuracy: {accuracy:.2f}")

        joblib.dump(self.model, 'irrigation_model.pkl')

    def predict_irrigation(self, moisture, temperature):
        current_hour = datetime.now().hour

        prediction = self.model.predict([[moisture, temperature, current_hour]])
        return prediction[0]


class CloudNotifier:
    def _init_(self):
        self.api_url = "https://api.exampleiot.com/notifications"
        self.api_key = "aditya_123#@1"

    def send_notification(self, message):
        payload = {
            "message": message,
            "recipient": "farmer_mobile_app",
            "timestamp": datetime.now().isoformat()
        }

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        }

        try:
            response = requests.post(
                self.api_url,
                data=json.dumps(payload),
                headers=headers
            )

            if response.status_code == 200:
                print("Notification sent successfully")
            else:
                print(f"Failed to send notification. Status code: {response.status_code}")
        except Exception as e:
            print(f"Error sending notification: {str(e)}")


class SmartAgricultureSystem:
    def _init_(self):
        self.sensor = SoilMoistureSensor()
        self.predictor = IrrigationPredictor()
        self.notifier = CloudNotifier()
        self.reading_interval = 3600

    def run(self):
        print("Smart Agriculture System Started")
        while True:
            moisture = self.sensor.read_moisture()
            temperature = self.sensor.read_temperature()

            print(f"\nCurrent Readings - Moisture: {moisture:.1f}%, Temperature: {temperature:.1f}°C")

            needs_irrigation = self.predictor.predict_irrigation(moisture, temperature)

            if needs_irrigation:
                message = f"Irrigation Alert! Soil moisture is low ({moisture:.1f}%). Temperature: {temperature:.1f}°C"
                print(message)
                self.notifier.send_notification(message)
            else:
                print("No irrigation needed at this time")

            time.sleep(self.reading_interval)


if _name_ == "_main_":
    system = SmartAgricultureSystem()
    system.run()